In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import nbconvert
import nbformat
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#GCPARKS IS The Master File for Guest Claims

gcparks = pd.read_excel('GCPARKS.xlsx')
attendance = pd.read_excel('Attendance.xlsx')
attendance = attendance.set_index('Unnamed: 0')

In [3]:
#Split Datasets into specific parks

grouped = gcparks.groupby('Personnel Area Desc')

dfs = {}

for division, subset in grouped:
    dfs[division] = subset

ecdf = dfs['0569 - EPCOT - FL']
hsdf = dfs['0663 - Disney\'s Hollywood Studios - FL']
mkdf = dfs['2092 - MAGIC KINGDOM 1029 - FL']
akdf = dfs['2227 - ANIMAL KINGDOM 1029 - FL']

In [4]:
#RATES

def calculate_incident_rates(df, location_name, attendance, fiscal_years):
    total_unique = df['Claim Number'].nunique()
    incident_claim_fy = df.groupby('Fiscal Year')['Claim Number'].nunique()

    claim_count = df[df['Claim Status'] != 'INCIDENT']['Claim Number'].nunique()

    incurred_year = df[df['Claim Status'] != 'INCIDENT'].groupby('Fiscal Year').agg({'Claim Number': 'nunique', 'Disney Claim Total Incurred': 'sum'})
    incurred_total = incurred_year['Disney Claim Total Incurred'].sum()

    values_for_fy = {}
    
    for fiscal_year in fiscal_years:
        value_for_fy = attendance.loc[fiscal_year, location_name]
        values_for_fy[fiscal_year] = value_for_fy
    
    incident_rates = {}
    
    for year in fiscal_years:
        attendance = values_for_fy.get(year, 0)
        incident_claims = incident_claim_fy.get(year, 0)
        
        incident_rate = (incident_claims / attendance * 1000000)
        
        incident_rates[f'{location_name.lower()}incidentrate{year[2:]}'] = incident_rate

    incident_rates_values = [incident_rates.get(f'{location_name.lower()}incidentrate{year[2:]}', 0) for year in fiscal_years]
    
    incident_rate_data = pd.DataFrame({
        'Location': location_name,
        'Fiscal Year': fiscal_years,
        'Incident Rate (per 1,000,000)': incident_rates_values
    })

    return incident_rate_data


# Define your locations, dataframes, and fiscal years
locations = ['Epcot', 'MK', 'DHS', 'DAK']
dataframes = [ecdf, mkdf, hsdf, akdf]
fiscal_years = ['FY18', 'FY19', 'FY20', 'FY21', 'FY22', 'FY23']

all_incident_rates = pd.DataFrame()  # Create an empty DataFrame to store all incident rates

for location, df in zip(locations, dataframes):
    incident_rate_data = calculate_incident_rates(df, location, attendance, fiscal_years)
    all_incident_rates = pd.concat([all_incident_rates, incident_rate_data], ignore_index=True)

In [5]:
Rates = all_incident_rates.pivot(index='Fiscal Year', columns='Location', values='Incident Rate (per 1,000,000)')
Rates['4 Park Avg'] = Rates.mean(axis=1)

Rates

Location,DAK,DHS,Epcot,MK,4 Park Avg
Fiscal Year,,,,,
FY18,297.301129,266.970000,257.392600,358.379208,295.010734
FY19,265.454469,259.496541,256.217036,324.552039,276.430021
FY20,285.619607,289.051429,259.260489,327.228636,290.290040
FY21,358.445358,328.127710,327.739926,403.947850,354.565211
FY22,337.308068,342.339030,388.167644,430.624878,374.609905
FY23,376.773461,402.521470,454.914406,498.679613,433.222237


In [6]:
dataframes = [ecdf, mkdf, hsdf, akdf]
park_names = ['Epcot', 'MK', 'DHS', 'DAK']
fiscal_years = ['FY18', 'FY19', 'FY20', 'FY21', 'FY22', 'FY23']

Incurred = pd.DataFrame(columns=['Fiscal Year'])

#Calculate the sum
for df, park in zip(dataframes, park_names):
    sum_data = df.groupby('Fiscal Year')['Disney Claim Total Incurred'].sum().reset_index()
    sum_data.rename(columns={'Disney Claim Total Incurred': f'{park} Incurred'}, inplace=True)
    
    Incurred = pd.merge(Incurred, sum_data, on='Fiscal Year', how='outer')

Incurred['Grand Total'] = Incurred.sum(axis=1)

Incurred

,Fiscal Year,Epcot Incurred,MK Incurred,DHS Incurred,DAK Incurred,Grand Total
0,FY18,2263194.34,3588952.51,917147.04,1158388.54,7927682.43
1,FY19,1095566.61,4821704.14,840041.05,689155.31,7446467.11
2,FY20,734352.14,2227075.82,881756.32,555570.68,4398754.96
3,FY21,823037.31,2525943.80,790497.04,732453.45,4871931.60
4,FY22,2891361.09,3402022.37,1045841.61,1549622.81,8888847.88
5,FY23,1109408.42,2816675.67,610792.74,737184.71,5274061.54
6,FY24,400.00,23600.00,4300.00,650.00,28950.00


In [7]:
####Exploratory Analysis


In [8]:
sorted_dataframes = []  # Initialize an empty list to store the sorted DataFrames

for i, dataframe in enumerate(dataframes):
    table = dataframe.groupby(['Location Code Desc', 'Event of Incident Desc','Source of Incident Desc']).size().reset_index(name='Count')
    sorted_table = table.sort_values(by='Count', ascending=False)
    sorted_dataframe = pd.DataFrame(sorted_table)  # Convert the sorted table to a DataFrame
    sorted_dataframes.append(sorted_dataframe)  # Append the sorted DataFrame to the list

In [9]:
MKCombos=sorted_dataframes[1]
MKCombos.head(n=10)

,Location Code Desc,Event of Incident Desc,Source of Incident Desc,Count
4509,Magic Kingdom Park,Fatigue / Illness,Person - Self,900
3135,Guest First Aid - Main Street,Fatigue / Illness,Person - Self,602
654,Be Our Guest Restaurant,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,453
6994,Space Mountain,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,436
2174,"Crystal Palace, The",Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,262
857,Big Thunder Mountain Railroad,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,221
1759,Cinderella's Royal Table,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,214
1984,Cosmic Ray's Starlight Cafe,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,202
6762,Seven Dwarfs Mine Train,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,193
5248,"Main Street, U.S.A.",Slip/Trip/Misstep - Trip,Walking Surface - Outdoor - Track/Rail,189


In [10]:
HSCombos=sorted_dataframes[2]
HSCombos.head(n=10)

,Location Code Desc,Event of Incident Desc,Source of Incident Desc,Count
3224,Rock 'n' Roller Coaster Starring Aerosmith,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,460
952,Disney's Hollywood Studios,Fatigue / Illness,Person - Self,452
1631,Guest First Aid - Hollywood Studios,Fatigue / Illness,Person - Self,244
4559,"Twilight Zone Tower of Terror, The",On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,224
3430,Sci-Fi Dine-In Theater Restaurant,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,218
3675,Star Tours,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,202
1717,Hollywood & Vine,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,191
3534,Slinky Dog Dash,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,166
2689,Millennium Falcon: Smugglers Run,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,145
2369,Mama Melrose's Ristorante Italiano,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,137


In [11]:
AKCombos=sorted_dataframes[3]
AKCombos.head(n=10)

,Location Code Desc,Event of Incident Desc,Source of Incident Desc,Count
1478,Expedition Everest - Legend of the Forbidden M...,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,874
2014,Flight of Passage,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,498
1185,Disney's Animal Kingdom Theme Park,Fatigue / Illness,Person - Self,366
2295,Guest First Aid - Discovery Island,Fatigue / Illness,Person - Self,195
2811,Kilimanjaro Safaris,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,177
4271,Tusker House Restaurant,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,162
785,Dinosaur,On Ride - Ride Motion,Attraction Related - Park Vehicle - Attraction...,152
1975,Flight of Passage,Fainting/Seizure - Fainting,Person - Self,139
3973,Satu’li Canteen,Eating/Drinking,Food/Drink and Related Items - Food Item/Beverage,118
1240,Disney's Animal Kingdom Theme Park,Slip/Trip/Misstep,Walking Surface,108


In [12]:
#Severity



In [13]:
sorted_dataframes = []  # Initialize an empty list to store the sorted DataFrames

for i, dataframe in enumerate(dataframes):
    table = dataframe[(dataframe['Fiscal Year'] == '2023')].groupby(['Location Code Desc', 'Event of Incident Desc', 'Source of Incident Desc']).size().reset_index(name='Count')
    sorted_table = table.sort_values(by='Count', ascending=False)
    sorted_dataframe = pd.DataFrame(sorted_table)  # Convert the sorted table to a DataFrame
    sorted_dataframes.append(sorted_dataframe)  # Append the sorted DataFrame to the list


In [1]:
attendance

NameError: name 'attendance' is not defined